In [4]:
import pandas as pd

# Caminho para o arquivo CSV
csv_file_path = '/home/rodrigo/Documentos/mestrado/GitHub/imagens_csv/swedish/train/leaf01/l1nr065_csv.csv'

# Ler o arquivo CSV
df = pd.read_csv(csv_file_path)

# Exibir as primeiras linhas do DataFrame
print(df.head())

# Exibir informações sobre o DataFrame
print(df.info())

Empty DataFrame
Columns: [3.32754703443878, 1.10450239943312, 0.495012203885435, 0.500588801777176, 1.41007000283006, 3.12507237853816, 3.32792570153061, 1.1013450173197, 0.493923647212069, 0.501817142438018, 1.40828391538473, 3.12315473709851, 3.38944913903061, 1.04462249684796, 0.512919215846191, 0.48689820562682, 1.41710341608271, 3.13964566077989, 9.04181281887755, 1.56279792467945, 0.473849056264551, 0.478604225814734, 2.52352144989082, 7.66219084099473, 9.21195392219388, 1.33398242823266, 0.491115207028657, 0.478862551279848, 2.49635769496553, 7.56358646015104, 6.58808992346939, 4.67982480335519, 0.517999483018753, 0.609463759387929, 1.89751521507695, 5.058197239537, 0.0333167368280491, 0.0333167368280492, 0.412675367052385, 0.0493070263513939, 0.076611721088429, 0.0773072239973983, 0.559646887319091, 0.130445666660977, 0.596302722802216, 0.600377678225707, 0.423863345411905, 0.563796328187535, 0.370345692937569, 0.366562765160135, 0.408315197837038, 0.379186711795365, 6.70296145

In [5]:
df

,3.32754703443878,1.10450239943312,0.495012203885435,0.500588801777176,1.41007000283006,3.12507237853816,3.32792570153061,1.1013450173197,0.493923647212069,0.501817142438018,...,0.509332209139714,0.399543815295845,21.1011247299779,19.1101869600435,24.8003368168046,28.8795548409694,485.167058509108,395.212696121942,639.991521346846,912.244262629499


In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponível!")
else:
    print("CUDA não está disponível.")


CUDA está disponível!
